In [1]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=1
%matplotlib inline

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import numpy as np                                                              
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf                                                         
import scipy as sp
import data.data_selector as ds                                                   
import utils.data_processing as dp
import utils.plot_functions as pf                                               
import analysis.analysis_picker as ap

/home/ryanchan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ryanchan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ryanchan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ryanchan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [3]:
class params(object):
  def __init__(self):
    self.model_type = "ica_subspace"
    self.model_name = "ica_subspace"
    self.version = "2"
analysis_params = params()
analysis_params.project_dir = (os.path.expanduser("~")+"/Work/Projects/")
analysis_params.model_dir = (analysis_params.project_dir+analysis_params.model_name)

In [4]:
import tensorflow as tf                                                                                          
import params.param_picker as pp                                                                                 
import models.model_picker as mp                                                                                 
import data.data_selector as ds  

In [7]:
model_type = "ica_subspace"
data_type = "vanhateren"

params = pp.get_params(model_type)
params.set_data_params(data_type)
params.num_patches = 500
params.batch_size = 500
params.rescale_data = False
params.center_data = True
data = ds.get_data(params)